In [1]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from collections import Counter
import json
import string
import networkx as nx
import matplotlib.pyplot as plt


In [2]:
%%time
# open file to read amazon product metadata 
fhr = open('data/amazon_data_file.txt', 'r', encoding='utf-8', errors='ignore')

# initialize a nested product dictionary that will hold cleaned up amazon product data
# key = ASIN; value = MetaData associated with ASIN
amazonProducts = {}

# read the data from the amazon-meta file;
# populate amazonProducts nested dicitonary;
ct=0

(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0, 0.0)
(time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
for line in fhr:
    if ct % 1000000 == 0:
        print("{} Done!".format(ct))
    ct+=1
    
    line = line.strip()
    # a product block started
    if(line.startswith("Id")):

        line = line.replace(" ","")
        Id = line.split(":")[1]
    
    elif(line.startswith("ASIN")):

        line = line.replace(" ","")
        ASIN = line.split(":")[1]
     
    elif(line.startswith("title")):
 
        Title = line.split(":")[1]
  
        Title = ' '.join(Title.split())
    elif(line.startswith("group")):
        Group = line.split(":")[1]
        Group = Group.replace(" ","")
 
    elif(line.startswith("salesrank")):
        line = line.replace(" ","")
        SalesRank = line.split(":")[1]

    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ([c for c in ls[2:]])
        
    elif(line.startswith("categories")):
        ls = line.split()
        cat_list = []
        for i in range(int(ls[1].strip())):
            temp = fhr.readline()
            temp = temp.lower()
            temp = temp.replace("|","-")
            temp = temp[4:]
            temp = temp.strip()
            cat_list.append(temp)
        
        Categories = cat_list

    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
        l1,l2,l3,l4,l5=[],[],[],[],[]
        for i in range(int(ls[4].strip())):
            temp = fhr.readline()
            temp = temp.split()
            l1.append(temp[0])
            l2.append(temp[2])
            l3.append(temp[4])
            l4.append(temp[6])
            l5.append(temp[8])
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull)=(l1,l2,l3,l4,l5)
        
            
    # a product block ended
    # write out fields to amazonProducts Dictionary
    elif (line==""):
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN]=MetaData
            MetaData['Id'] = Id            
            MetaData['title'] = Title
            MetaData['categories'] = Categories
            MetaData['group'] = Group
            MetaData['co-purchased'] = Copurchased
            MetaData['sales_rank'] = int(SalesRank)
            MetaData['total_reviews'] = int(TotalReviews)
            MetaData['avg_rating'] = float(AvgRating)
            MetaData['review_time_stamps_list']    = time_stamp
            MetaData['review_ASINs_list']          = review_ASINs
            MetaData['review_ratings_list']        = review_ratings
            MetaData['review_votes_list']          = review_votes
            MetaData['review_found_helpfull_list'] = review_helpfull
            
            
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0,0.0)
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
fhr.close()


df = pd.DataFrame(amazonProducts)
df = df.transpose()
temp = list(df['Id'].values)
df['Id'] = df.index
df.rename(columns={"Id":"ASIN"},inplace=True)
df.index = temp
new_col = {"ASIN":"product_id","review_ASINs_list":"user_ids"}
df = df.rename(new_col,axis=1)
df.head()

0 Done!
1000000 Done!
2000000 Done!
3000000 Done!
4000000 Done!
Wall time: 58.7 s


,product_id,title,categories,group,co-purchased,sales_rank,total_reviews,avg_rating,review_time_stamps_list,user_ids,review_ratings_list,review_votes_list,review_found_helpfull_list
0,0771044445,,[],,[],0,0,0.0,[],[],[],[],[]
1,0827229534,Patterns of Preaching,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0804215715, 156101074X, 0687023955, 068707423...",396585,2,5.0,"[2000-7-28, 2003-12-14]","[A2JW67OY8U6HHK, A2VE83MZF98ITY]","[5, 5]","[10, 6]","[9, 5]"
2,0738700797,Candlemas,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0738700827, 1567184960, 1567182836, 073870052...",168596,12,4.5,"[2001-12-16, 2002-1-7, 2002-1-24, 2002-1-28, 2...","[A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM...","[5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 1, 5]","[5, 5, 8, 4, 16, 5, 6, 8, 8, 5, 13, 1]","[4, 5, 8, 4, 16, 5, 6, 8, 5, 5, 9, 1]"
3,0486287785,World War II Allied Fighter Planes Trading Cards,[books[283155]-subjects[1000]-home & garden[48...,Book,[],1270652,1,5.0,[2003-7-10],[A3IDGASRQAW8B2],[5],[2],[2]
4,0842328327,Life Application Bible Commentary,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0842328130, 0830818138, 0842330313, 084232861...",631289,1,4.0,[2004-8-19],[A2591BUPXCS705],[4],[1],[1]


In [3]:
%%time
def parse_date(date_list):
    new_list=[]
    for date in date_list:
        x = date.split("-")
        year,month,day = int(x[0]),int(x[1]),int(x[2])
        new_list.append((year,month,day))
    return new_list

def clean_category(cat_id_list):
    new_list=[]
    for row  in cat_id_list:
        temp=[]
        x = row.split("-")
        for cid in x:
            if "[" in cid:
                y = cid.split("[")
                cat  = y[0]
                ID  = (y[1][:-1])
            else:
                cat = cid
                ID = -1
            temp.append((cat,ID))
        new_list.append(temp)
    return new_list       

df['product_id']  = df['product_id'].astype('str')
df['title'] = df['title'].apply(lambda x:re.sub(r'[^a-zA-Z0-9 ]','',x))
df['title'] = df['title'].str.lower()
df['categories'] = df['categories'].apply(clean_category)
df['group'] = df['group'].str.lower()
df['group'] = df['group'].str.replace(' ','')
df['group']  = df['group'].astype('str')
df['co-purchased']= df['co-purchased'].apply(lambda x: [str(i) for i in x])
df['sales_rank'] = df['sales_rank'].astype('int')
df['total_reviews'] = df['total_reviews'].astype('int')
df['avg_rating'] = df['avg_rating'].astype('float')
df['review_time_stamps_list'] = df['review_time_stamps_list'].apply(parse_date)
df['user_ids']= df['user_ids'].apply(lambda x: [str(i) for i in x])
df['review_ratings_list']= df['review_ratings_list'].apply(lambda x: [int(i) for i in x])
df['review_votes_list']= df['review_votes_list'].apply(lambda x: [int(i) for i in x])
df['review_found_helpfull_list']= df['review_found_helpfull_list'].apply(lambda x: [int(i) for i in x])
df['num_reviews'] = df['review_ratings_list'].apply(lambda x:len(x))
df.head()

Wall time: 35.7 s


,product_id,title,categories,group,co-purchased,sales_rank,total_reviews,avg_rating,review_time_stamps_list,user_ids,review_ratings_list,review_votes_list,review_found_helpfull_list,num_reviews
0,0771044445,,[],,[],0,0,0.0,[],[],[],[],[],0
1,0827229534,patterns of preaching,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0804215715, 156101074X, 0687023955, 068707423...",396585,2,5.0,"[(2000, 7, 28), (2003, 12, 14)]","[A2JW67OY8U6HHK, A2VE83MZF98ITY]","[5, 5]","[10, 6]","[9, 5]",2
2,0738700797,candlemas,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0738700827, 1567184960, 1567182836, 073870052...",168596,12,4.5,"[(2001, 12, 16), (2002, 1, 7), (2002, 1, 24), ...","[A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM...","[5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 1, 5]","[5, 5, 8, 4, 16, 5, 6, 8, 8, 5, 13, 1]","[4, 5, 8, 4, 16, 5, 6, 8, 5, 5, 9, 1]",12
3,0486287785,world war ii allied fighter planes trading cards,"[[(books, 283155), (subjects, 1000), (home & g...",book,[],1270652,1,5.0,"[(2003, 7, 10)]",[A3IDGASRQAW8B2],[5],[2],[2],1
4,0842328327,life application bible commentary,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0842328130, 0830818138, 0842330313, 084232861...",631289,1,4.0,"[(2004, 8, 19)]",[A2591BUPXCS705],[4],[1],[1],1


In [52]:
df_book

,product_id,title,categories,group,co-purchased,sales_rank,total_reviews,avg_rating,review_time_stamps_list,user_ids,review_ratings_list,review_votes_list,review_found_helpfull_list,num_reviews
2,0738700797,candlemas,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0738700827, 1567184960, 1567182836, 073870052...",168596,12,4.5,"[(2001, 12, 16), (2002, 1, 7), (2002, 1, 24), ...","[A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM...","[5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 1, 5]","[5, 5, 8, 4, 16, 5, 6, 8, 8, 5, 13, 1]","[4, 5, 8, 4, 16, 5, 6, 8, 5, 5, 9, 1]",12
6,0486220125,how the other half lives,"[[(books, 283155), (subjects, 1000), (arts & p...",book,"[0486401960, 0452283612, 0486229076, 071484034...",188784,17,4.0,"[(1997, 7, 4), (1998, 10, 11), (1999, 4, 15), ...","[ATVPDKIKX0DER, AUEZ7NVOEHYRY, ATVPDKIKX0DER, ...","[5, 5, 5, 4, 5, 5, 5, 2, 5, 1, 4, 4, 4, 5, 4, ...","[12, 13, 18, 14, 17, 23, 7, 6, 23, 8, 7, 3, 7,...","[11, 12, 7, 10, 11, 16, 6, 0, 21, 8, 1, 2, 4, ...",17
8,0231118597,losing matt shepard,"[[(books, 283155), (subjects, 1000), (gay & le...",book,"[B000067D0Y, 0375727191, 080148605X, 156023257...",277409,15,4.5,"[(2000, 10, 31), (2000, 11, 1), (2000, 11, 19)...","[A2F1X6YFCJZ1FH, A1OZQCZAK21S6M, AL5D52NA8F67F...","[5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2, 5]","[10, 13, 16, 0, 6, 5, 3, 6, 10, 1, 8, 7, 10, 1...","[9, 12, 13, 0, 6, 5, 3, 6, 10, 1, 8, 5, 4, 5, 2]",15
12,1590770218,fantastic food with splenda,"[[(books, 283155), (subjects, 1000), (cooking,...",book,"[0871319640, 0399530258, 1590770536, 158040197...",24741,12,4.5,"[(2004, 4, 21), (2004, 6, 6), (2004, 7, 5), (2...","[A2V9UBVMQFDV20, A3NXQLHXJZO5FB, A1RNV50D6DNE4...","[5, 5, 1, 5, 5, 5, 5, 4, 5, 5, 1, 5]","[8, 4, 7, 13, 6, 5, 11, 8, 7, 5, 4, 2]","[8, 4, 3, 13, 5, 4, 10, 4, 4, 4, 1, 2]",12
32,0761116982,staff meals from chanterelle,"[[(books, 283155), (subjects, 1000), (cooking,...",book,"[1579652395, 0393020436, 0767903609, 031229023...",150657,11,4.5,"[(2000, 12, 31), (2001, 2, 22), (2001, 3, 11),...","[A2I9I77KMEBA4S, A1ANWZQ24OC744, A2NF3S0T3TRSM...","[4, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5]","[50, 14, 15, 4, 27, 12, 19, 33, 4, 2, 4]","[48, 8, 14, 4, 26, 11, 19, 32, 4, 2, 4]",11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548472,0312868146,necroscope,"[[(books, 283155), (subjects, 1000), (horror, ...",book,"[0812553632, 0812536282, 0812520629, 081252061...",496637,21,4.0,"[(1999, 6, 20), (1999, 7, 3), (1999, 7, 14), (...","[ATVPDKIKX0DER, AEB45TKHZI37F, A3GZZ7O20BZQ94,...","[3, 5, 5, 3, 4, 5, 1, 5, 5, 4, 3, 5, 4, 4, 5, ...","[0, 0, 0, 0, 0, 5, 1, 0, 0, 0, 2, 1, 8, 8, 3, ...","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 1, 6, 6, 2, ...",21
548486,0688164668,california pizza kitchen pasta salads soups an...,"[[(books, 283155), (subjects, 1000), (cooking,...",book,"[0028609883, 0963203401, 0312206321, 080925730...",23466,11,4.0,"[(1999, 11, 18), (2000, 4, 29), (2002, 5, 25),...","[AXKOOVT9RTE6O, A2TD3DAJ7CNI2T, A1B99SOTMR8MLI...","[3, 5, 5, 3, 4, 4, 4, 3, 4, 4, 5]","[36, 23, 1, 0, 0, 3, 4, 2, 5, 2, 1]","[28, 23, 1, 0, 0, 3, 4, 1, 5, 2, 1]",11
548487,0972356207,diario della studentessa jean,"[[(books, 283155), (subjects, 1000), (biograph...",book,"[0486261808, 0486280918, 0972356215, 007142089...",139511,18,4.5,"[(2002, 12, 3), (2002, 12, 9), (2002, 12, 10),...","[AWTU0HBSEGLVY, AWHX6ZSA0UKMF, A3E91O9LXUR70H,...","[5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 2, 2, 0, 2, 3, 5, 5, 5, 2, 5, 2, 6, 5, 6, ...","[0, 2, 2, 0, 1, 3, 5, 5, 5, 2, 5, 2, 6, 5, 6, ...",18
548508,158483000X,needless casualties of war,"[[(books, 283155), (subjects, 1000), (religion...",book,"[1584830492, 096290497X, 0768422329, 158483037...",34810,11,5.0,"[(1999, 11, 23), (2000, 3, 3), (2000, 7, 29), ...","[A2IKNL1OQ0HOP4, A3GSI8Q25U5438, A1MH4RM7O2GDY...","[4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5]","[13, 28, 9, 7, 11, 16, 33, 13, 25, 6, 5]","[12, 25, 7, 7, 10, 15, 33, 10, 24, 6, 5]",11


##### only considering books that has received atlest 10 reviews

In [4]:
df_book = df.loc[(df['group']=='book') & (df['num_reviews']>10)]
df_book.shape

(63296, 14)

In [5]:
print(len(set(df_book['product_id'])))

63296


In [67]:
def get_code_set(code_list):
    code_set = set()
    for i in code_list:
        for j in i:
            code_set.add(j[1])
    return code_set
            
def get_cat_similarity_code(A,B):
    code_A = get_code_set(A)
    code_B = get_code_set(B)
    num = len(code_A.intersection(code_B))
    den = len(code_A.union(code_B))
    if den:
        return num/den
    else:
        return 0
print("sample_testing")
a = df_book['categories'].iloc[110]
b = df_book['categories'].iloc[11]
print(get_cat_similarity_code(a,b))

def get_int_list(l):
    nl = []
    for x in l:
        nl.append(int(x))
    return nl
def get_edge_weights(r1,r2,v1,v2,h1,h2):
    r1 = get_int_list(r1)
    r2 = get_int_list(r2)
    v1 = get_int_list(v1)
    v2 = get_int_list(v2)
    h1 = get_int_list(h1)
    h2 = get_int_list(h2)
    l1 = len(r1)
    l2 = len(r2)
    val1=0
    c1 = []
    for i in range(l1):
        a,b=0,0
        if len(v1)==0 or sum(v1)==0:
            a=0
        if len(h1)==0 or sum(h1)==0:
            b=0
        else:
            if i>=len(v1):
                a=0
            if i>=len(h1):
                b=0
            else:
                a = (v1[i])/(sum(v1))
                b = (h1[i])/(sum(h1))
        c1.append(a+b)
    for i in range(l1):
        val1+=(r1[i])*(c1[i])
    val2=0
    c2 = []
    for i in range(l2):
        a,b=0,0
        if len(v2)==0 or sum(v2)==0:
            a=0
        if len(h2)==0 or sum(h2)==0:
            b=0
        else:
            if i>=len(v2):
                a=0
            if i>=len(h2):
                b=0
            else:
                a = (v2[i])/(sum(v2))
                b = (h2[i])/(sum(h2))
        c2.append(a+b)
    for i in range(l2):
        val2+=(r2[i])*c2[i]
    return (val1/sum(r1),val2/sum(r2)) 

sample_testing
0.10344827586206896


In [16]:
%%time
t=[]
for  cat in df_book['co-purchased']:
    for x in cat:
        t.append(x)

nodes = set(df_book['product_id']).intersection(set(t))
print("number of nodes:",len(nodes))
t1 = set(t)
edges=0
for  cat in df_book['co-purchased']:
    for x in cat:
        if x in t1:
            edges+=1
print("number of edges:",edges)  

number of nodes: 26885
number of edges: 298580
Wall time: 320 ms


In [68]:
r1,v1,h1 = amazonProducts['0738700797']['review_ratings_list'],amazonProducts['0738700797']['review_votes_list'],amazonProducts['0738700797']['review_found_helpfull_list']
r2,v2,h2 = amazonProducts['0688164668']['review_ratings_list'],amazonProducts['0688164668']['review_votes_list'],amazonProducts['0688164668']['review_found_helpfull_list'] 

In [69]:
get_edge_weights(r1,r2,v1,v2,h1,h2)

(0.15587044534412955, 0.1766442114035697)

#### amazonBooks dictionary

In [65]:
%%time
node_list = set(df_book['product_id'])
amazonBooks = {}
for row in df_book[['product_id','co-purchased']].iterrows():
        source_node= row[1][0]
        amazonBooks[source_node] = {}
        co_prod = row[1][1]
        amazonBooks[source_node]['co-purchased'] = ([cp for cp in co_prod if cp in node_list])
#         amazonBooks[source_node]['rating_list'] = [rl for rl in rating_list]
#         amazonBooks[source_node]['vote_list'] = [vl for rl in vote_list]
#         amazonBooks[source_node]['helpful_list'] = [hl for rl in helpful_list]

Wall time: 3.59 s


In [84]:
%%time
copurchaseGraph = nx.Graph()
ct =0
for source_node,co_prod_dict in amazonBooks.items():
    ct+=1
    if ct %100==0:
        print("{} completed !".format(ct))
    co_prod = co_prod_dict['co-purchased']
    if len(co_prod):
        copurchaseGraph.add_node(source_node)
    for target_node in co_prod:
        copurchaseGraph.add_node(target_node)
        similarity = 0        
        n1 = (clean_category(amazonProducts[source_node]['categories']))
        n2 = (clean_category(amazonProducts[target_node]['categories']))
        r1,v1,h1 = amazonProducts[source_node]['review_ratings_list'],amazonProducts[source_node]['review_votes_list'],amazonProducts[source_node]['review_found_helpfull_list']
        r2,v2,h2 = amazonProducts[target_node]['review_ratings_list'],amazonProducts[target_node]['review_votes_list'],amazonProducts[target_node]['review_found_helpfull_list'] 
        similarity = get_cat_similarity_code(n1,n2)
#         val1,val2 = get_edge_weights(r1,r2,v1,v2,h1,h2)
        copurchaseGraph.add_edge(source_node, target_node,weight=similarity)

100 completed !
200 completed !
300 completed !
400 completed !
500 completed !
600 completed !
700 completed !
800 completed !
900 completed !
1000 completed !
1100 completed !
1200 completed !
1300 completed !
1400 completed !
1500 completed !
1600 completed !
1700 completed !
1800 completed !
1900 completed !
2000 completed !
2100 completed !
2200 completed !
2300 completed !
2400 completed !
2500 completed !
2600 completed !
2700 completed !
2800 completed !
2900 completed !
3000 completed !
3100 completed !
3200 completed !
3300 completed !
3400 completed !
3500 completed !
3600 completed !
3700 completed !
3800 completed !
3900 completed !
4000 completed !
4100 completed !
4200 completed !
4300 completed !
4400 completed !
4500 completed !
4600 completed !
4700 completed !
4800 completed !
4900 completed !
5000 completed !
5100 completed !
5200 completed !
5300 completed !
5400 completed !
5500 completed !
5600 completed !
5700 completed !
5800 completed !
5900 completed !
6000 c

47100 completed !
47200 completed !
47300 completed !
47400 completed !
47500 completed !
47600 completed !
47700 completed !
47800 completed !
47900 completed !
48000 completed !
48100 completed !
48200 completed !
48300 completed !
48400 completed !
48500 completed !
48600 completed !
48700 completed !
48800 completed !
48900 completed !
49000 completed !
49100 completed !
49200 completed !
49300 completed !
49400 completed !
49500 completed !
49600 completed !
49700 completed !
49800 completed !
49900 completed !
50000 completed !
50100 completed !
50200 completed !
50300 completed !
50400 completed !
50500 completed !
50600 completed !
50700 completed !
50800 completed !
50900 completed !
51000 completed !
51100 completed !
51200 completed !
51300 completed !
51400 completed !
51500 completed !
51600 completed !
51700 completed !
51800 completed !
51900 completed !
52000 completed !
52100 completed !
52200 completed !
52300 completed !
52400 completed !
52500 completed !
52600 comp

In [85]:
%%time
dc = nx.degree(copurchaseGraph)
for asin in nx.nodes(copurchaseGraph):
    metadata = amazonBooks[asin]
    metadata['degree_centrality'] = int(dc[asin])
    ego = nx.ego_graph(copurchaseGraph, asin, radius=1)
    metadata['clustering_coeff'] = round(nx.average_clustering(ego),2)
    amazonBooks[asin] = metadata

#write copurchaseGraph data to file
fhw=open("amazon-books-copurchase.edgelist_simple",'wb')
nx.write_weighted_edgelist(copurchaseGraph, fhw)
fhw.close()

Wall time: 37.1 s


In [148]:
fhr=open("amazon-books-copurchase.edgelist_simple", 'rb')
copurchaseGraph=nx.read_weighted_edgelist(fhr)
fhr.close()

# Now let's assume a person is considering buying the following book;
# what else can we recommend to them based on copurchase behavior 
# we've seen from other users?
print ("Looking for Recommendations for Customer Purchasing this Book:")
print ("--------------------------------------------------------------")
#'0231118597'
purchasedAsin = '0738700797'

# Let's first get some metadata associated with this book
print ("ASIN = ", purchasedAsin) 
print ("Title = ", amazonProducts[purchasedAsin]['title'])
print ("SalesRank = ", amazonProducts[purchasedAsin]['sales_rank'])
print ("TotalReviews = ", amazonProducts[purchasedAsin]['total_reviews'])
print ("AvgRating = ", amazonProducts[purchasedAsin]['avg_rating'])
print ("DegreeCentrality = ", amazonBooks[purchasedAsin]['degree_centrality'])
print ("ClusteringCoeff = ", amazonBooks[purchasedAsin]['clustering_coeff'])

Looking for Recommendations for Customer Purchasing this Book:
--------------------------------------------------------------
ASIN =  0738700797
Title =  Candlemas
SalesRank =  168596
TotalReviews =  12
AvgRating =  4.5
DegreeCentrality =  4
ClusteringCoeff =  1.0


* Now let's look at the ego network associated with purchasedAsin in the copurchaseGraph - which is esentially comprised of all the books  that have been copurchased with this book in the past Get the depth-1 ego network of purchasedAsin from copurchaseGraph, and assign the resulting graph to purchasedAsinEgoGraph.

In [142]:
%%time
purchasedAsinEgoGraph = nx.ego_graph(copurchaseGraph,purchasedAsin,radius=1)
print ('Purchased Asin Ego Graph:',
       'Number of Nodes= ', purchasedAsinEgoGraph.number_of_nodes(),
       'Number of Edges= ', purchasedAsinEgoGraph.number_of_edges())

Purchased Asin Ego Graph: Number of Nodes=  7 Number of Edges=  8
Wall time: 1.99 ms


* Next, recall that the edge weights in the copurchaseGraph is a measure of the similarity between the books connected by the edge. So we can use the  island method to only retain those books that are highly simialr to the  purchasedAsin Use the island method on purchasedAsinEgoGraph to only retain edges with  threshold >= 0.5, and assign resulting graph to purchasedAsinEgoTrimGraph

In [143]:
%%time
threshold = 0.4
purchasedAsinEgoTrimGraph = nx.Graph()

for f, t, e in purchasedAsinEgoGraph.edges(data=True):
    if e['weight'] > threshold:
        purchasedAsinEgoTrimGraph.add_edge(f,t,weight=e['weight'])

print('Purchased_Asin_Ego_Trim_Graph:',
         'threshold=', threshold,
         'Nodes=', purchasedAsinEgoTrimGraph.number_of_nodes(),
         'Edges=', purchasedAsinEgoTrimGraph.number_of_edges())

Purchased_Asin_Ego_Trim_Graph: threshold= 0.4 Nodes= 7 Edges= 8
Wall time: 975 µs


* Next, recall that given the purchasedAsinEgoTrimGraph you constructed above, you can get at the list of nodes connected to the purchasedAsin by a single  hop (called the neighbors of the purchasedAsin)  Find the list of neighbors of the purchasedAsin in the   purchasedAsinEgoTrimGraph, and assign it to purchasedAsinNeighbors

In [144]:
# try:
purchasedAsinNeighbors = list(purchasedAsinEgoTrimGraph.neighbors((purchasedAsin)))
print("Neighbors of {}: is".format(purchasedAsin),(purchasedAsinNeighbors))
print("Number of trimmed neighbours of ",purchasedAsin,"=",len(purchasedAsinNeighbors))
# except nx.exception.NetworkXError:
#     print("ekfjvb svdc")
    

Neighbors of 0875420915: is ['1567184960', '0892813695', '0892818670', '0875420907', '1567186637', '0722532717']
Number of trimmed neighbours of  0875420915 = 6


* Next, let's pick the Top Five book recommendations from among the  purchasedAsinNeighbors based on one or more of the following data of the  neighboring nodes: SalesRank, AvgRating, TotalReviews, DegreeCentrality,  and ClusteringCoeff

* Note that, given an asin, you can get at the metadata associated with   it using amazonBooks (similar to lines 49-56 above).
* Now, come up with a composite measure to make Top Five book  recommendations based on one or more of the following metrics associated  with nodes in purchasedAsinNeighbors: SalesRank, AvgRating,  TotalReviews, DegreeCentrality, and ClusteringCoeff 

In [145]:
N,NR=0,0
for i in purchasedAsinNeighbors:
    NR=NR+(amazonProducts[i]['total_reviews']*amazonProducts[i]['avg_rating'])
    N=N+amazonProducts[i]['total_reviews']

ranking=[]
for i in purchasedAsinNeighbors:
    temp=(NR+(amazonProducts[i]['total_reviews']*amazonProducts[i]['avg_rating']))/(N+amazonProducts[i]['total_reviews'])
    ranking.append([i,round(temp,4)])
ranking.sort(key=lambda x:x[1])
ranking = ranking[::-1] 
recommendations=ranking[:5]
print(recommendations)

[['0722532717', 4.1677], ['1567186637', 4.131], ['0892818670', 4.0693], ['0892813695', 4.0693], ['0875420907', 4.0674]]


In [146]:
# Top 5 recommendations 
#(ASIN, and associated Title, Sales Rank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff)
for i in range(len(recommendations)):
    print()
    print ("ASIN = ", recommendations[i][0]) 
    print ("Title = ", amazonProducts[recommendations[i][0]]['title'])
    print ("SalesRank = ", amazonProducts[recommendations[i][0]]['sales_rank'])
    print ("TotalReviews = ", amazonProducts[recommendations[i][0]]['total_reviews'])
    print ("AvgRating = ", amazonProducts[recommendations[i][0]]['avg_rating'])
    print ("DegreeCentrality = ", amazonBooks[recommendations[i][0]]['degree_centrality'])
    print ("ClusteringCoeff = ", amazonBooks[recommendations[i][0]]['clustering_coeff'])


ASIN =  0722532717
Title =  Wicca
SalesRank =  278403
TotalReviews =  35
AvgRating =  4.5
DegreeCentrality =  3
ClusteringCoeff =  0.58

ASIN =  1567186637
Title =  The Sabbats
SalesRank =  112190
TotalReviews =  19
AvgRating =  4.5
DegreeCentrality =  2
ClusteringCoeff =  1.0

ASIN =  0892818670
Title =  The Pagan Book of Days
SalesRank =  77251
TotalReviews =  11
AvgRating =  4.0
DegreeCentrality =  6
ClusteringCoeff =  0.57

ASIN =  0892813695
Title =  The Pagan Book of Days
SalesRank =  486306
TotalReviews =  11
AvgRating =  4.0
DegreeCentrality =  4
ClusteringCoeff =  0.7

ASIN =  0875420907
Title =  Ancient Ways
SalesRank =  118006
TotalReviews =  15
AvgRating =  4.0
DegreeCentrality =  3
ClusteringCoeff =  0.83


In [147]:
amazonProducts[purchasedAsin]

{'Id': '218868',
 'title': 'Wheel of the Year',
 'categories': ['books[283155]-subjects[1000]-religion & spirituality[22]-earth-based religions[12472]-wicca[12484]',
  'books[283155]-subjects[1000]-religion & spirituality[22]-earth-based religions[12472]-witchcraft[12486]',
  'books[283155]-subjects[1000]-religion & spirituality[22]-new age[12621]-general[12667]',
  'books[283155]-subjects[1000]-religion & spirituality[22]-occult[12689]-magic[12710]',
  'books[283155]-subjects[1000]-religion & spirituality[22]-earth-based religions[12472]-paganism[297501]'],
 'group': 'Book',
 'co-purchased': ['1567186637',
  '0875420907',
  '1567184960',
  '0738700525',
  '0738700827'],
 'sales_rank': 52852,
 'total_reviews': 15,
 'avg_rating': 4.5,
 'review_time_stamps_list': ['1997-3-18',
  '1998-10-18',
  '1999-10-31',
  '1999-11-20',
  '2001-7-1',
  '2001-8-13',
  '2001-11-26',
  '2002-4-18',
  '2003-1-8',
  '2003-8-15',
  '2003-8-15',
  '2003-12-15',
  '2004-2-17',
  '2004-2-22',
  '2004-9-26'],


In [123]:
#set(amazonProducts[purchasedAsin]['co-purchased']).intersection(set([pid[0] for pid in recommendations]))

{'1567182836', '1567184960'}

In [254]:
def recommendations_engine(purchasedAsin,threshold = 0.5):
    purchasedAsinEgoGraph = nx.ego_graph(copurchaseGraph,purchasedAsin,radius=1)
    purchasedAsinEgoTrimGraph = nx.Graph()

    for f, t, e in purchasedAsinEgoGraph.edges(data=True):
        if e['weight'] > threshold:
            purchasedAsinEgoTrimGraph.add_edge(f,t,weight=e['weight'])
    try:
        purchasedAsinNeighbors = list(purchasedAsinEgoTrimGraph.neighbors(purchasedAsin))
        N,NR=0,0
        for i in purchasedAsinNeighbors:
            NR=NR+(amazonProducts[i]['total_reviews']*amazonProducts[i]['avg_rating'])
            N=N+amazonProducts[i]['total_reviews']

        ranking=[]
        for i in purchasedAsinNeighbors:
            temp=(NR+(amazonProducts[i]['total_reviews']*amazonProducts[i]['avg_rating']))/(N+amazonProducts[i]['total_reviews'])
            ranking.append([i,round(temp,4)])
        ranking.sort(key=lambda x:x[1])
        ranking = ranking[::-1] 
        recommendations=ranking[:5]
        return recommendations
    except nx.exception.NetworkXError:
        #print("exception occured")
        return []
 

In [264]:
%%time
reco_list=[]
ct =0
for f, t, e in copurchaseGraph.edges(data=True):
    if ct%10000==0:
        print("{} completed!".format(ct))
    ct+=1
    rcmd = recommendations_engine(f)# reccomendation for source node as target node are already co-product
    tpl = (f,rcmd)
    reco_list.append(tpl)

0 completed!
10000 completed!
20000 completed!
30000 completed!
40000 completed!
50000 completed!
60000 completed!
70000 completed!
80000 completed!
90000 completed!
100000 completed!
110000 completed!
Wall time: 2min 46s


In [265]:
book_reco_df = pd.DataFrame(reco_list,columns=['product_id','reco_list'])

In [266]:
book_reco_df.to_csv("book_reco.csv",index=False)

In [154]:
ans=[]
for i in range(10):
    for x in recommendations:
        path = nx.shortest_path_length(copurchaseGraph, source=purchasedAsin, target=x[0])
        ans.append(path)
print(sum(ans)/len(ans))


2.8
